In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

# modelization 
import sklearn
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
## system
import glob, os

In [16]:
train_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/X_Train_concat_y_train.gzip"
test_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/X_Test_merged_clean.gzip"
base_for_test_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Test/Baselines/Baseline_forecast_test.csv"
base_for_train_path = "C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Train/Baselines/Baseline_forecast_train.csv"

In [3]:
def Read_Parquet(chemin):
  return pd.read_parquet(chemin, engine='fastparquet')

In [41]:
train = Read_Parquet(train_path)
train.head()

,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,humidity_max,...,precip_percentile_25,precip_percentile_75,precip_skew,precip_kurtosis,Month,season,lat,lon,height_sta,Ground_truth
Index,,,,,,,,,,,,,,,,,,,,,
14066001_0,276.91,282.59,280.333750,281.325,278.5100,282.1750,-0.445114,-1.418811,79.9,98.1,...,0.0,0.00,4.587317,19.043478,1,winter,49.334,-0.431,2.0,3.4
14066001_1,280.65,284.37,282.936667,282.870,282.1925,283.9625,-0.204054,-0.728304,71.8,92.0,...,0.0,0.05,2.768227,7.524247,1,winter,49.334,-0.431,2.0,11.7
14066001_10,277.91,281.33,279.557917,279.520,279.1575,280.1750,0.097627,-0.208982,81.4,92.6,...,0.0,0.40,2.517940,5.760873,1,winter,49.334,-0.431,2.0,1.0
14066001_100,275.53,287.57,282.112917,283.950,276.8575,285.5100,-0.439847,-1.378881,56.1,90.3,...,0.0,0.00,4.410995,17.920294,4,winter,49.334,-0.431,2.0,5.6
14066001_101,279.29,285.41,282.805000,282.720,282.2225,284.0125,-0.378186,-0.411371,76.8,95.2,...,0.0,0.00,2.538396,5.073702,4,winter,49.334,-0.431,2.0,3.2


In [42]:
test = Read_Parquet(test_path)
test.set_index("Index", inplace=True)
test.head()

,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,humidity_max,...,precip_median,precip_percentile_25,precip_percentile_75,precip_skew,precip_kurtosis,Month,season,lat,lon,height_sta
Index,,,,,,,,,,,,,,,,,,,,,
14047002_100,281.86,283.24,282.565000,282.515,282.2875,282.835,-0.000790,-0.932193,88.36,98.46,...,0.000,0.0,0.45,1.483913,0.656080,12,winter,49.275,-0.712,60.0
14047002_223,283.69,286.43,285.415000,285.400,284.9400,285.905,-0.519703,-0.350026,75.69,96.88,...,0.000,0.0,0.00,3.989829,14.892857,12,winter,49.275,-0.712,60.0
14047002_236,282.42,284.47,283.064167,282.990,282.6450,283.215,1.358096,1.388465,79.28,93.26,...,0.000,0.0,0.30,2.813251,8.290722,12,winter,49.275,-0.712,60.0
14047002_256,278.88,286.40,283.611250,284.625,282.3075,285.575,-0.792405,-0.848946,77.21,96.76,...,0.000,0.0,0.00,2.514334,5.405663,12,winter,49.275,-0.712,60.0
14047002_277,278.35,283.85,282.006000,283.040,280.4550,283.405,-0.869940,-0.830229,87.36,94.17,...,0.165,0.0,0.65,1.414487,0.573291,12,winter,49.275,-0.712,60.0


In [43]:
test.dtypes

temp_min                         float64
temp_max                         float64
temp_mean                        float64
temp_median                      float64
temp_percentile_25               float64
temp_percentile_75               float64
temp_skew                        float64
temp_kurtosis                    float64
humidity_min                     float64
humidity_max                     float64
humidity_mean                    float64
humidity_median                  float64
humidity_percentile_25           float64
humidity_percentile_75           float64
humidity_skew                    float64
humidity_kurtosis                float64
dew_point_temp_min               float64
dew_point_temp_max               float64
dew_point_temp_mean              float64
dew_point_temp_median            float64
dew_point_temp_percentile_25     float64
dew_point_temp_percentile_75     float64
dew_point_temp_skew              float64
dew_point_temp_kurtosis          float64
Vx_min          

## Encoding categorical features

In [44]:
train_enc = pd.get_dummies(train,prefix_sep='_')
train_enc.head()

,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,humidity_max,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Index,,,,,,,,,,,,,,,,,,,,,
14066001_0,276.91,282.59,280.333750,281.325,278.5100,282.1750,-0.445114,-1.418811,79.9,98.1,...,0,0,0,0,0,0,0,0,0,1
14066001_1,280.65,284.37,282.936667,282.870,282.1925,283.9625,-0.204054,-0.728304,71.8,92.0,...,0,0,0,0,0,0,0,0,0,1
14066001_10,277.91,281.33,279.557917,279.520,279.1575,280.1750,0.097627,-0.208982,81.4,92.6,...,0,0,0,0,0,0,0,0,0,1
14066001_100,275.53,287.57,282.112917,283.950,276.8575,285.5100,-0.439847,-1.378881,56.1,90.3,...,0,0,0,0,0,0,0,0,0,1
14066001_101,279.29,285.41,282.805000,282.720,282.2225,284.0125,-0.378186,-0.411371,76.8,95.2,...,0,0,0,0,0,0,0,0,0,1


In [45]:
test_enc = pd.get_dummies(test,prefix_sep='_')
test_enc.head()

,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,humidity_max,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Index,,,,,,,,,,,,,,,,,,,,,
14047002_100,281.86,283.24,282.565000,282.515,282.2875,282.835,-0.000790,-0.932193,88.36,98.46,...,0,0,0,0,0,1,0,0,0,1
14047002_223,283.69,286.43,285.415000,285.400,284.9400,285.905,-0.519703,-0.350026,75.69,96.88,...,0,0,0,0,0,1,0,0,0,1
14047002_236,282.42,284.47,283.064167,282.990,282.6450,283.215,1.358096,1.388465,79.28,93.26,...,0,0,0,0,0,1,0,0,0,1
14047002_256,278.88,286.40,283.611250,284.625,282.3075,285.575,-0.792405,-0.848946,77.21,96.76,...,0,0,0,0,0,1,0,0,0,1
14047002_277,278.35,283.85,282.006000,283.040,280.4550,283.405,-0.869940,-0.830229,87.36,94.17,...,0,0,0,0,0,1,0,0,0,1


## Forecast meteo france models

In [46]:
base_for_train = pd.read_csv(base_for_train_path)
base_for_train.drop(['number_sta','date'], axis=1, inplace=True)
base_for_train.rename(columns={'Prediction':'Prediction_meteo_france'}, inplace=True)
base_for_train.head()

,Prediction_meteo_france,Id
0,5.027973,86118001_0
1,0.957647,86149001_0
2,21.019514,56081003_0
3,4.260686,53215001_0
4,17.256493,22135001_0


In [47]:
base_for_test = pd.read_csv(base_for_test_path)
base_for_test.rename(columns={'Prediction':'Prediction_meteo_france'}, inplace=True)
base_for_test.head()

,Prediction_meteo_france,Id
0,12.921875,86118001_149
1,2.186354,86149001_149
2,0.036655,56081003_149
3,0.024249,53215001_149
4,0.377269,22135001_149


In [49]:
test_enc.reset_index(inplace=True)
test_enc.rename(columns={'Index':'Id'}, inplace=True)
test_enc.head()

,Id,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
0,14047002_100,281.86,283.24,282.565000,282.515,282.2875,282.835,-0.000790,-0.932193,88.36,...,0,0,0,0,0,1,0,0,0,1
1,14047002_223,283.69,286.43,285.415000,285.400,284.9400,285.905,-0.519703,-0.350026,75.69,...,0,0,0,0,0,1,0,0,0,1
2,14047002_236,282.42,284.47,283.064167,282.990,282.6450,283.215,1.358096,1.388465,79.28,...,0,0,0,0,0,1,0,0,0,1
3,14047002_256,278.88,286.40,283.611250,284.625,282.3075,285.575,-0.792405,-0.848946,77.21,...,0,0,0,0,0,1,0,0,0,1
4,14047002_277,278.35,283.85,282.006000,283.040,280.4550,283.405,-0.869940,-0.830229,87.36,...,0,0,0,0,0,1,0,0,0,1


In [50]:
test= pd.merge(base_for_test, test_enc, how='right').set_index("Id")
test.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Id,,,,,,,,,,,,,,,,,,,,,
14047002_100,4.361040,281.86,283.24,282.565000,282.515,282.2875,282.835,-0.000790,-0.932193,88.36,...,0,0,0,0,0,1,0,0,0,1
14047002_223,0.000000,283.69,286.43,285.415000,285.400,284.9400,285.905,-0.519703,-0.350026,75.69,...,0,0,0,0,0,1,0,0,0,1
14047002_236,4.294544,282.42,284.47,283.064167,282.990,282.6450,283.215,1.358096,1.388465,79.28,...,0,0,0,0,0,1,0,0,0,1
14047002_256,8.192842,278.88,286.40,283.611250,284.625,282.3075,285.575,-0.792405,-0.848946,77.21,...,0,0,0,0,0,1,0,0,0,1
14047002_277,7.473648,278.35,283.85,282.006000,283.040,280.4550,283.405,-0.869940,-0.830229,87.36,...,0,0,0,0,0,1,0,0,0,1


In [51]:
train_enc.reset_index(inplace=True)
train_enc.rename(columns={'Index':'Id'}, inplace=True)
train= pd.merge(base_for_train, train_enc, how='right').set_index("Id")
train.head()

,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,humidity_min,...,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter
Id,,,,,,,,,,,,,,,,,,,,,
14066001_0,5.545129,276.91,282.59,280.333750,281.325,278.5100,282.1750,-0.445114,-1.418811,79.9,...,0,0,0,0,0,0,0,0,0,1
14066001_1,7.372426,280.65,284.37,282.936667,282.870,282.1925,283.9625,-0.204054,-0.728304,71.8,...,0,0,0,0,0,0,0,0,0,1
14066001_10,7.151256,277.91,281.33,279.557917,279.520,279.1575,280.1750,0.097627,-0.208982,81.4,...,0,0,0,0,0,0,0,0,0,1
14066001_100,1.914063,275.53,287.57,282.112917,283.950,276.8575,285.5100,-0.439847,-1.378881,56.1,...,0,0,0,0,0,0,0,0,0,1
14066001_101,0.581547,279.29,285.41,282.805000,282.720,282.2225,284.0125,-0.378186,-0.411371,76.8,...,0,0,0,0,0,0,0,0,0,1


## Kfold for cross validation 

In [52]:
def training_kfold(df, kfold): ## stratified
    df.loc[:,"kfold"] = -1
    KfoldsDf_train = df.reset_index().sample(frac=1)
    kf = model_selection.KFold(n_splits=kfold)
    for f, (t_, v_) in enumerate(kf.split(X=KfoldsDf_train)): 
        KfoldsDf_train.loc[v_, 'kfold'] = f 
    return KfoldsDf_train

In [53]:
## chosing 10 folds for cross validation
KfoldsDf_train = training_kfold(train, kfold=10)

## Training

In [30]:
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb

In [54]:
KfoldsDf_train.head()

,Id,Prediction_meteo_france,temp_min,temp_max,temp_mean,temp_median,temp_percentile_25,temp_percentile_75,temp_skew,temp_kurtosis,...,Month_8,Month_9,Month_10,Month_11,Month_12,season_fall,season_spring,season_summer,season_winter,kfold
133579,78354001_665,NaN,280.11,288.51,285.188333,285.480,283.9800,287.230,-0.422980,-0.744274,...,0,0,1,0,0,1,0,0,0,8
73982,41152001_162,1.808574,288.10,291.56,289.406667,289.015,288.3425,290.345,0.513951,-1.146613,...,0,0,0,0,0,0,0,1,0,4
113706,61387001_110,0.229429,279.38,290.34,285.370417,286.555,281.1800,288.355,-0.454446,-1.279116,...,0,0,0,0,0,0,0,0,1,7
110688,61006005_575,1.221831,289.92,298.75,293.719583,293.060,290.8950,295.985,0.172964,-1.343390,...,0,0,0,0,0,0,0,1,0,6
59483,36173002_697,1.035465,277.63,280.75,279.236667,279.420,277.9750,280.085,-0.200085,-1.325138,...,0,0,0,1,0,1,0,0,0,3


In [38]:
KfoldsDf_train.columns

Index(['Id', 'Prediction_meteo_france', 'temp_min', 'temp_max', 'temp_mean',
       'temp_median', 'temp_percentile_25', 'temp_percentile_75', 'temp_skew',
       'temp_kurtosis', 'humidity_min', 'humidity_max', 'humidity_mean',
       'humidity_median', 'humidity_percentile_25', 'humidity_percentile_75',
       'humidity_skew', 'humidity_kurtosis', 'dew_point_temp_min',
       'dew_point_temp_max', 'dew_point_temp_mean', 'dew_point_temp_median',
       'dew_point_temp_percentile_25', 'dew_point_temp_percentile_75',
       'dew_point_temp_skew', 'dew_point_temp_kurtosis', 'Vx_min', 'Vx_max',
       'Vx_mean', 'Vx_median', 'Vx_percentile_25', 'Vx_percentile_75',
       'Vx_skew', 'Vx_kurtosis', 'Vy_min', 'Vy_max', 'Vy_mean', 'Vy_median',
       'Vy_percentile_25', 'Vy_percentile_75', 'Vy_skew', 'Vy_kurtosis',
       'precip_sum', 'precip_min', 'precip_max', 'precip_mean',
       'precip_median', 'precip_percentile_25', 'precip_percentile_75',
       'precip_skew', 'precip_kurtosis', 'Mo

In [26]:
# !pip install xgboost 

In [28]:
xgboost = xgb.XGBRegressor(n_estimators=100,reg_lambda=1,max_depth=23, n_jobs=-1,reg_alpha=3, min_child_weight= 3, gamma=5, learning_rate=0.01, colsample_bytree=1.0)

In [55]:
def run(KfoldsDf_train, fold, model, metric):
    # all columns are features except id, target and kfold columns    
    # get training data using folds 
    # #data.set_index("Id", inplace=True)
    # df_train = data[data.kfold != fold].reset_index(drop=True) 
    
    # # get validation data using folds 
    # df_valid = data[data.kfold == fold].reset_index(drop=True) 
    # get training data 
    y_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=fold]['Ground_truth'].values
    y_val = KfoldsDf_train[KfoldsDf_train["kfold"]==fold]['Ground_truth'].values
    X_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=fold].drop(['kfold',"Ground_truth"], axis=1).set_index('Id')
    X_val =KfoldsDf_train[KfoldsDf_train["kfold"]==fold].drop(['kfold',"Ground_truth"], axis=1).set_index('Id')

    
    #model  = ensemble.RandomForestRegressor(n_jobs=-1,random_state=2021)
    
    # fit model on training data (ohe) 
    model.fit(X_train,y_train)
 
    
    # predict on validation data 
    # we need the probability values as we are calculating AUC 
    # we will use the probability of 1s 
    valid_preds = model.predict(X_val) 
    valid_preds += 1
    y_val +=1
    score = metric(y_val, valid_preds) 
    model_name = type(model).__name__
    
    return score, model_name, fold,model

In [56]:
if __name__ == "__main__": 
    errors = []
    models =  []
    for i in range(10):
        score, model_name, fold,model = run(KfoldsDf_train, i,  xgboost ,mean_absolute_percentage_error)
        print(f"The model : {model_name} has a MAPE of {score} in fold {fold}")

        errors.append(score)
        models.append(model)
    

    print(f"The mean score of all the folds is {np.mean(errors)}")

The model : XGBRegressor has a score of 0.38482829999021706 in fold 0
The model : XGBRegressor has a score of 0.36655743578468847 in fold 1
The model : XGBRegressor has a score of 0.3619640048063081 in fold 2
The model : XGBRegressor has a score of 0.37537101870746076 in fold 3
The model : XGBRegressor has a score of 0.3456105664864863 in fold 4
The model : XGBRegressor has a score of 0.3551778309267101 in fold 5
The model : XGBRegressor has a score of 0.36878759438609293 in fold 6
The model : XGBRegressor has a score of 0.3939000169529502 in fold 7
The model : XGBRegressor has a score of 0.370048872309373 in fold 8
The model : XGBRegressor has a score of 0.3339742549392051 in fold 9
The mean score of all the folds is 0.36562198952894914


## Hyper parametres tuning

In [57]:
# !pip install optuna 

  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=3b4a7f0405f737a9e78c4333198de1055708953b17a031493e726100ea959ea1
  Stored in directory: c:\users\alaek\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built pyperclip


In [58]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
import joblib

In [59]:
xgboost = xgb.XGBRegressor(n_estimators=100,reg_lambda=1,max_depth=23, n_jobs=-1,reg_alpha=3, min_child_weight= 3, gamma=5, learning_rate=0.01, colsample_bytree=1.0)

Help on XGBRegressor in module xgboost.sklearn object:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  XGBRegressor(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'reg:squarederror', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost regression.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of gradient boosted trees.  Equivalent to number of boosting
 |          rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      learning_rate : Optional[float]
 |          Boosting learning rate (xgb's "eta")
 |      verbosity : Optional[int]
 |          The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |      objective : typing.Union[str, typing.Callable[[numpy.ndarray, numpy.ndarray], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]
 |          Specify the learning t

In [63]:
def objective(trial: Trial,X,y) -> float:
    
    joblib.dump(study, 'study.pkl')

    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
                'max_depth':trial.suggest_int('max_depth', 2, 25),
                'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
                'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
                'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
                'gamma':trial.suggest_int('gamma', 0, 5),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
                'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
                'n_jobs' : -1,
                'objective':'reg:pseudohubererror',
                'gpu_id':0,
            }
    
    model = xgb.XGBRegressor(**param)
    
    return cross_val_score(model, X, y, cv=10).mean()

In [64]:
y_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=fold]['Ground_truth'].values
X_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=fold].drop(['kfold',"Ground_truth"], axis=1).set_index('Id')

study = optuna.create_study(direction='minimize',sampler=TPESampler(), )
study.optimize(lambda trial : objective(trial,X_train,y_train),n_trials= 10)

[I 2022-01-14 09:12:51,365] A new study created in memory with name: no-name-533282bf-cb89-40ef-af64-60007891aff4
[I 2022-01-14 09:27:21,103] Trial 0 finished with value: -1.839448960840263e+72 and parameters: {'n_estimators': 385, 'max_depth': 18, 'reg_alpha': 5, 'reg_lambda': 0, 'min_child_weight': 0, 'gamma': 0, 'learning_rate': 0.4915377482344181, 'colsample_bytree': 0.4}. Best is trial 0 with value: -1.839448960840263e+72.


In [ ]:
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
visualization.plot_optimization_history(study)

In [ ]:
visualization.plot_slice(study)

In [ ]:
visualization.plot_parallel_coordinate(study)

In [ ]:
visualization.plot_contour(study)

In [23]:
!pip install wandb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.1.0 requires scikit-image>=0.16.1, which is not installed.



  Using cached yaspin-2.1.0-py3-none-any.whl (18 kB)
  Using cached subprocess32-3.5.4-py3-none-any.whl
  Using cached shortuuid-1.0.8-py3-none-any.whl (9.5 kB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached PyYAML-6.0-cp39-cp39-win_amd64.whl (151 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl


In [24]:
import wandb
wandb.login()

ENTITY ='Respira'# ENTER Your team name here if you have one.

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alaekichi (use `wandb login --relogin` to force relogin)


In [57]:
# import wandb and the lightgbm callback
import wandb
from wandb.lightgbm import log_summary, wandb_callback
import lightgbm as lgb

# initialize a new wandb project
wandb.init()

y_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=0]['Ground_truth']
y_val = KfoldsDf_train[KfoldsDf_train["kfold"]==0]['Ground_truth']
X_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=0].drop(['kfold',"Ground_truth"], axis=1).set_index('index')
X_val =KfoldsDf_train[KfoldsDf_train["kfold"]==0].drop(['kfold',"Ground_truth"], axis=1).set_index('index')

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['huber'],
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbosity': -1
}
wandb.config.update(params)

# train
# add lightgbm callback
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                valid_names=('validation'),
                callbacks=[wandb_callback(),
                           lgb.early_stopping(stopping_rounds=25)])

# predict
y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
# eval

y_pred+=1
y_val+=1
print('The MAPE of prediction is:', mean_absolute_percentage_error(y_val, y_pred))

# log feature importance and model checkpoint 
# log_summary(gbm, save_model_checkpoint=True)

iteration,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
validation_huber,█▇▇▆▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁
iteration,19


[1]	validation's huber: 1.9793
Training until validation scores don't improve for 25 rounds.
[2]	validation's huber: 1.95884
[3]	validation's huber: 1.94189
[4]	validation's huber: 1.92655
[5]	validation's huber: 1.91239
[6]	validation's huber: 1.89848
[7]	validation's huber: 1.88539
[8]	validation's huber: 1.87391
[9]	validation's huber: 1.86215
[10]	validation's huber: 1.8513
[11]	validation's huber: 1.84119
[12]	validation's huber: 1.83215
[13]	validation's huber: 1.82281
[14]	validation's huber: 1.81414
[15]	validation's huber: 1.80511
[16]	validation's huber: 1.797
[17]	validation's huber: 1.79
[18]	validation's huber: 1.78241
[19]	validation's huber: 1.77592
[20]	validation's huber: 1.76938
Did not meet early stopping. Best iteration is:
[20]	validation's huber: 1.76938
The MAPE of prediction is: 1.0855728870438448


In [58]:
mean_absolute_percentage_error(y_val,y_pred)

1.0855728870438448

In [59]:
sklearn.metrics.r2_score(y_val,y_pred)

0.14488891185830421

In [61]:
# initialize wandb run
wandb.init(project="xgboost-training_defi_ia")

y_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=0]['Ground_truth'].values
y_val = KfoldsDf_train[KfoldsDf_train["kfold"]==0]['Ground_truth'].values
X_train = KfoldsDf_train[KfoldsDf_train["kfold"]!=0].drop(['kfold',"Ground_truth"], axis=1).set_index('index')
X_val =KfoldsDf_train[KfoldsDf_train["kfold"]==0].drop(['kfold',"Ground_truth"], axis=1).set_index('index')

xg_reg = xgb.XGBRegressor(#objective ='reg:pseudohubererror'
    colsample_bytree = 0.3, 
    learning_rate = 0.1,
    max_depth = 5, 
    alpha = 10, 
    n_estimators = 10,)

xg_reg.fit(X_train,y_train,callbacks=[wandb.xgboost.wandb_callback()])

preds = xg_reg.predict(X_val)
preds+=1
y_val+=1

mape = mean_absolute_percentage_error(y_val, y_pred)
print("MAPE: %f" % (mape))
wandb.log({"MAPE": mape})

[10:52:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAPE: 1.085573


In [83]:
import wandb
wandb.init(sync_tensorboard=True)

In [86]:
if __name__ == "__main__": 
    errors = []
    # wandb.init(project="xgboost-training_defi_ia")

    for i in range(10):
        score, model_name, fold = run(KfoldsDf_train, i,  xgboost ,mean_absolute_percentage_error)
        print(f"The model : {model_name} has a score of {score} in fold {fold}")

        errors.append(score)
    

    print(f"The mean score of all the folds is {np.mean(errors)}")

[11:23:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The model : XGBRegressor has a score of 0.5694052793522024 in fold 0
[11:32:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The model : XGBRegressor has a score of 0.5088792230102418 in fold 1
[11:40:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The model : XGBRegressor has a score of 0.5883234498410976 in fold 2
[11:48:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The model : XGBRegressor has a score of 0.5299367537103495 in fold 3
[11:56:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The model : XGBRegressor has a score of 0.45904987087833626 in fold 4
[12:04:58] WARNING: